In [68]:
import requests
import pandas as pd
import os
from random import random
import time

### Run in the terminal: python3.6 api_server.py

## Test read from the machine

In [73]:
resp = requests.get('http://146.139.66.23:5000/', json={
  "setting": ["R501:PHASEB"],
  "fcup": [],
  "bpm": []
})

In [74]:
print(type(resp.json()))
#df = pd.DataFrame.from_dict(resp.json())
df = pd.json_normalize(resp.json())
print(df)

<class 'dict'>
                         time  setting.control.R501:PHASEB  \
0  2022-10-26 15:10:56.783873                        325.0   

   setting.monitor.R501:PHASEB  
0                        325.0  


In [107]:
#phase scan
setting_list = ["R501:PHASEB","R501:CONTROL_AMP"]
degree = 0.0
max_degree = 10
amplitude = 2.0
dict_post = {"R501:CONTROL_AMP" : amplitude}
print("Dictionary Post: ",dict_post)
resp_post = requests.post('http://146.139.66.23:5000/', json=dict_post)

for i in range(max_degree):
    degree = 95 + i
    dict_post = {"R501:PHASEB" : degree}
    print("Dictionary Post: ",dict_post)
    resp_post = requests.post('http://146.139.66.23:5000/', json=dict_post)
    #degree += 1
    time.sleep(5)
    for names in setting_list:
        resp = requests.get('http://146.139.66.23:5000/', json={
                "setting": [names],
                "fcup": [],
                "bpm": [] })
        df = pd.json_normalize(resp.json())
        temp_cntrl_name = "setting.control." + names
        temp_mon_name = "setting.monitor." + names
        print("Monitor: ",temp_mon_name,df[temp_mon_name].iat[0])
        print("Control: ",temp_cntrl_name,df[temp_mon_name].iat[0])
        dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_cntrl_name,df[temp_cntrl_name].iat[0])
        os.system(dbwrite)
        dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_mon_name,df[temp_mon_name].iat[0])
        os.system(dbwrite)

Dictionary Post:  {'R501:CONTROL_AMP': 2.0}
Dictionary Post:  {'R501:PHASEB': 95}
Monitor:  setting.monitor.R501:PHASEB 95.0
Control:  setting.control.R501:PHASEB 95.0
Monitor:  setting.monitor.R501:CONTROL_AMP 0.0
Control:  setting.control.R501:CONTROL_AMP 0.0
Dictionary Post:  {'R501:PHASEB': 96}
Monitor:  setting.monitor.R501:PHASEB 96.0
Control:  setting.control.R501:PHASEB 96.0
Monitor:  setting.monitor.R501:CONTROL_AMP 0.0
Control:  setting.control.R501:CONTROL_AMP 0.0
Dictionary Post:  {'R501:PHASEB': 97}
Monitor:  setting.monitor.R501:PHASEB 97.0
Control:  setting.control.R501:PHASEB 97.0
Monitor:  setting.monitor.R501:CONTROL_AMP 0.0
Control:  setting.control.R501:CONTROL_AMP 0.0
Dictionary Post:  {'R501:PHASEB': 98}
Monitor:  setting.monitor.R501:PHASEB 98.0
Control:  setting.control.R501:PHASEB 98.0
Monitor:  setting.monitor.R501:CONTROL_AMP 0.0
Control:  setting.control.R501:CONTROL_AMP 0.0
Dictionary Post:  {'R501:PHASEB': 99}
Monitor:  setting.monitor.R501:PHASEB 99.0
Con

In [108]:
dict_post = {"R501:PHASEB" : 95}
print("Dictionary Post: ",dict_post)
resp_post = requests.post('http://146.139.66.23:5000/', json=dict_post)
dict_post = {"R501:CONTROL_AMP" : 2}
print("Dictionary Post: ",dict_post)
resp_post = requests.post('http://146.139.66.23:5000/', json=dict_post)

Dictionary Post:  {'R501:PHASEB': 95}
Dictionary Post:  {'R501:CONTROL_AMP': 2}


## Start of good code to run loops to collect data

In [113]:
#settings
write2database = True #write to influx?
post_settings = True #whether to send new magnet settings
setting_list = ["QTA800:CONTROL_Y1","QTA800:CONTROL_X1","QTA800:CONTROL_Y2"]#,"STA801:CONTROL_X","STA801:CONTROL_Y"] #which magnets to read
#setting_list = ["R501:PHASEB","R501:CONTROL_AMP"]
max_vals = [10.0,15.0,20.0,1.5,1.5] #max field setting for bl magnets
min_vals = [0.5,0.5,0.5,-1.5,-1.5]
off_vals = [0.0,0.0,0.0,-1.1,-1.1]
delta_vals = [0.5,0.5,0.5,0.02,0.02] #amount by which to change 
orig_vals = [4.051278,5.904756,5.272509,-1.036212,0.827078] #initial tune settings before scans
num_loops = 120 #finite numer of samples if using
num_samples = 2 #times to measure the quads before changing them # not used at present

## Loop that will change current setting by small amounts

In [114]:
#right now changes one element at a time then reads
for loops in range(num_loops):
    print("On Loop Number: ",loops)
    i=0
    for names in setting_list:
    #print(names)
        #read present setting
        if (post_settings):
            resp = requests.get('http://146.139.66.23:5000/', json={
            "setting": [names],
            "fcup": [],
            "bpm": [] })
            df = pd.json_normalize(resp.json())
            temp_cntrl_name = "setting.control." + names
            temp_mon_name = "setting.monitor." + names
            new = 0.0
            delta = 0.0
            #calculate new setting and post it
            delta=random()*delta_vals[i]#could be modified
            if random() < 0.55:
                new = df[temp_mon_name].iat[0] + delta #use monitor value
            else:
                new = df[temp_mon_name].iat[0] - delta #use monitor value
            #check to make sure within min/max
            if new > max_vals[i] or new < min_vals[i]:
                new = orig_vals[i]
            if loops%40 == 0:
                new = orig_vals[i]
            dict_post = {names : new}
            print("Dictionary Post: ",dict_post)
            # uncomment below for actual post!!!
            resp_post = requests.post('http://146.139.66.23:5000/', json=dict_post)
            i=i+1
            time.sleep(2) #wait after post
        if (write2database):
            for names in setting_list:
                #read new settings
                resp = requests.get('http://146.139.66.23:5000/', json={
                "setting": [names],
                "fcup": [],
                "bpm": [] })
                df = pd.json_normalize(resp.json())
                temp_cntrl_name = "setting.control." + names
                temp_mon_name = "setting.monitor." + names
                print("Monitor: ",temp_mon_name,df[temp_mon_name].iat[0])
                dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_cntrl_name,df[temp_cntrl_name].iat[0])
                os.system(dbwrite)
                dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_mon_name,df[temp_mon_name].iat[0])
                os.system(dbwrite)
    time.sleep(1) #time to see detector rates

On Loop Number:  0
Dictionary Post:  {'QTA800:CONTROL_Y1': 4.051278}
Monitor:  setting.monitor.QTA800:CONTROL_Y1 4.717944145202637
Monitor:  setting.monitor.QTA800:CONTROL_X1 5.963364124298096
Monitor:  setting.monitor.QTA800:CONTROL_Y2 4.620494365692139
Dictionary Post:  {'QTA800:CONTROL_X1': 5.904756}
Monitor:  setting.monitor.QTA800:CONTROL_Y1 4.717944145202637
Monitor:  setting.monitor.QTA800:CONTROL_X1 5.963364124298096
Monitor:  setting.monitor.QTA800:CONTROL_Y2 4.620494365692139
Dictionary Post:  {'QTA800:CONTROL_Y2': 5.272509}
Monitor:  setting.monitor.QTA800:CONTROL_Y1 4.036625862121582
Monitor:  setting.monitor.QTA800:CONTROL_X1 5.868125915527344
Monitor:  setting.monitor.QTA800:CONTROL_Y2 5.213900566101074
On Loop Number:  1
Dictionary Post:  {'QTA800:CONTROL_Y1': 3.8608800253546462}
Monitor:  setting.monitor.QTA800:CONTROL_Y1 4.036625862121582
Monitor:  setting.monitor.QTA800:CONTROL_X1 5.868125915527344
Monitor:  setting.monitor.QTA800:CONTROL_Y2 5.213900566101074
Dictiona

In [115]:
#to restore orig values
setting_list = ["QTA800:CONTROL_Y1","QTA800:CONTROL_X1","QTA800:CONTROL_Y2"]
orig_vals = [4.498,6.125,5.360,-1.036212,0.827078] #initial tune settings before scans
i=0
for names in setting_list:
    dict_post = {names : orig_vals[i]}
    print("Dictionary Post: ",dict_post)
    resp_post = requests.post('http://146.139.66.23:5000/', json=dict_post)
    i=i+1

Dictionary Post:  {'QTA800:CONTROL_Y1': 4.498}
Dictionary Post:  {'QTA800:CONTROL_X1': 6.125}
Dictionary Post:  {'QTA800:CONTROL_Y2': 5.36}


## Original loop that sets random values to each setting

In [ ]:
#settings
write2database = True #write to influx?
post_settings = True #whether to send new magnet settings
setting_list = ["QTA800:CONTROL_Y1","QTA800:CONTROL_X1","QTA800:CONTROL_Y2"] #which magnets to read
max_vals = [10.0,15.0,20.0,2.2,2.2] #max field setting for bl magnets
min_vals = [0.0,0.0,0.0,-1.1,-1.1]
off_vals = [0.0,0.0,0.0,-1.1,-1.1]
num_loops = 20000 #finite numer of samples if using
num_samples = 2 #times to measure the quads before changing them

In [ ]:
for loops in range(num_loops):
    print("On Loop Number: ",loops)
    i=0
    for names in setting_list:
        #print(names)
        if (post_settings):
            #dict_post = {names : 10.0}
            dict_post = {names : random()*max_vals[i] + off_vals[i]}
            #print(dict_post)
            resp_post = requests.post('http://146.139.66.23:5000/', json=dict_post)
            i=i+1
    time.sleep(2)
    for samples in range(num_samples):
        for names in setting_list:
        #print(names)
            resp = requests.get('http://146.139.66.23:5000/', json={
                "setting": [names],
                "fcup": [],
                "bpm": [] })
            df = pd.json_normalize(resp.json())
            temp_cntrl_name = "setting.control." + names
            temp_mon_name = "setting.monitor." + names
            #print(df[temp_cntrl_name].iat[0])
            #print(df[temp_mon_name].iat[0])
            if (write2database):
                dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_cntrl_name,df[temp_cntrl_name].iat[0])
                os.system(dbwrite)
                dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_mon_name,df[temp_mon_name].iat[0])
                os.system(dbwrite)
        time.sleep(0.5)